## Refer to the Speech to Text 3.0 Swagger documentation: https://develop.cris.ai/swagger/ui/index?urls.primaryName=Speech%20Services%20API%20v3.0#

In [ ]:
from azure.storage.blob import BlobServiceClient

In [ ]:
import requests
import json
import pandas as pd
import argparse
import time
import http.client, urllib.request, urllib.parse, urllib.error, base64

In [ ]:
## Speech to Text region
region = "----------"

In [ ]:
## Speech to Text - subscription key
SUBSCRIPTION_KEY = "--------------"

In [ ]:
## Storage Account connection string and SAS key
connect_str = '-----------------'
SAS = '----------------------------'

In [ ]:
## storage account - account name and container name. This where we store the audio files
container_name = "--------"
account_name = "------------"

In [ ]:
## Speech to Text Endpoint for Transcriptions 

BaseURL ="https://" + region + ".api.cognitive.microsoft.com/speechtotext/v3.0/transcriptions/"


In [ ]:
## Storage path
STORAGE_PATH="https://" + account_name + ".blob.core.windows.net/" + container_name + "/"

## Prepare HTTP POST Request Body with associated Speech to Text properties

In [ ]:
## Speech to Text - transcription properties
def bulkTranscription(blob_uri, blob_name):
    BD = """
            {
                "contentUrls": [ 
                    "__URL__"],
                "locale": "en-US",
                "displayName": "blob_name",
                "properties": {
                    "punctuationMode": "Automatic",
                    "profanityFilterMode": "Masked",
                    "wordLevelTimestampsEnabled": "False",
                    "diarizationEnabled" : "True"
                }
            }
            """
    BD = BD.replace("__URL__", URL)  
    BD = BD.replace("blob_name", blobname)
    return BD

## Initialize HTTP POST request headers 

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}


## Initialize the connection to our Blob storage account and container. Store the list of blobs in blob_list

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs()


## For every blob (audio file) in our storage account container call the Speech to Text REST API and transcribe the audio file

In [ ]:
for blob in blob_list:
    print("\t" + blob.name)
    RECORDINGS_BLOB_URI = STORAGE_PATH + blob.name + SAS
    URL = RECORDINGS_BLOB_URI
    blobname = blob.name
    r = requests.post(BaseURL, headers=headers,data=bulkTranscription(URL,blobname))
    print(r.text)
    js = json.loads(r.text)
    print(js)

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
r = requests.get(BaseURL, headers=headers)
js = json.loads(r.text)

In [ ]:
print(js)

## List all transcriptions in either of the following states (Not Started, Running, Succeeded, Failed)

In [ ]:
def listAllTranscriptions() :
    import datetime
    df = pd.DataFrame(columns=["id","name", "status","createdDateTime","lastActionDateTime","CallDuration"])
    headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
    r = requests.get(BaseURL, headers=headers)
    js = json.loads(r.text)
    #print(js)
    for TR in js['values']:
        if TR["status"] not in ("Succeeded")  :
            Dur="-"
        else :
            Dur = TR["properties"]["duration"]
        df = df.append({
         "id":                  TR["self"].split('/')[-1],
         "name":                TR["displayName"],
         "status":              TR["status"],
         "createdDateTime":     datetime.datetime.strptime(TR["createdDateTime"],"%Y-%m-%dT%H:%M:%SZ"),            
         "lastActionDateTime":  datetime.datetime.strptime(TR["lastActionDateTime"],"%Y-%m-%dT%H:%M:%SZ"),
         "CallDuration":            Dur      
          }, ignore_index=True)
        df["ProcTime"] = df["lastActionDateTime"] - df["createdDateTime"]
        
    return(df)

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
r = requests.get(BaseURL, headers=headers)

In [ ]:
listAllTranscriptions()

In [ ]:
transcriptions = listAllTranscriptions()


## Go through each transcription that finished and is in "Succeeded" status and obtain the json output of the transcription. Save the json file locally

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY,
}

params = urllib.parse.urlencode({
    # Request parameters
})
number_of_rows = len(transcriptions)
for index, row in transcriptions.iterrows():
    if row["status"] in ("Succeeded"):
        conn = http.client.HTTPSConnection(region + '.api.cognitive.microsoft.com')
        conn.request("GET", "/speechtotext/v3.0/transcriptions/" + row["id"] + "/files?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        contentUrl = json.loads(data)['values'][0]['links']['contentUrl']
        r = requests.get(contentUrl, headers=headers)
        json_text = json.loads(r.text)
        file_name = row["name"].split('.')[0] + ".json"
        with open(file_name, 'w') as f:
            json.dump(json_text, f)

## Truncate the transcriptions log 

In [ ]:
def deleteAllTranscriptions() :
    trs=listAllTranscriptions()
    for x in trs["id"] :
        print(x)
        headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
        r = requests.delete(BaseURL+x, headers=headers)
        print(r)

In [ ]:
deleteAllTranscriptions() 

## Obtain sentiment scores using Text Analytics for each of the utterances from the transcription

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
r = requests.get(json.loads(data)['values'][0]['links']['contentUrl'], headers=headers)
json_text = json.loads(r.text)

In [ ]:
json_text['combinedRecognizedPhrases'][0]['display']

In [ ]:
TEXT_SUBSCRIPTION_KEY = "-----------------------------"

In [ ]:
text_headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': TEXT_SUBSCRIPTION_KEY,
}

text_params = urllib.parse.urlencode({
    # Request parameters
})

docs = {"documents": [
           {
               "language": "en",
               "id" : "1",
               "text": json_text['combinedRecognizedPhrases'][0]['display']
               
           }
]}


try:
    conn = http.client.HTTPSConnection(region + '.api.cognitive.microsoft.com')
    conn.request("POST", "/text/analytics/v3.1-preview.1/sentiment?%s" % text_params, json.dumps(docs), text_headers)
    response = conn.getresponse()
    data = response.read()
    
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [ ]:
json.loads(data)